In [ ]:
!pip install pyhive
!pip install thrift
!pip install thrift-sasl

### Establishing Hive connection

In [ ]:
from pyhive import hive

# Replace the placeholders with your actual details
host = '35.192.106.136'
port = 10000  # default port
username = 'knagendra1729@cluster-1729-m'  # your Hive username
database = 'default'  # optional

# Create a connection
conn = hive.Connection(host=host, port=port, username=username, database=database)




In [ ]:
# Create a cursor
cursor = conn.cursor()

cursor.execute('Describe hive_table2')
schema = cursor.fetchall()
print(schema)

[('full_name', 'string', ''), ('age', 'int', ''), ('martial_status', 'string', ''), ('email', 'string', ''), ('phone', 'string', ''), ('full_address', 'string', ''), ('job_title', 'string', ''), ('membership_date', 'timestamp', '')]


#### Hive table data count

In [ ]:
cursor.execute('SELECT * FROM hive_table2')
result = cursor.fetchall()
hive_count = len(result)
print(hive_count)

500


In [ ]:
cursor.execute('select * from hive_table2 limit 10')
result = cursor.fetchall()
print(result)


[('annaliese etoile', 52, 'married', 'aetoile2w@artisteer.com', '814-2985', '"43 Nova Circle', 'Garden Grove', None), ('carlos machin', 41, 'separated', 'cmachinj0@xinhuanet.com', '215-833-2589', '"91 Anzinger Alley', 'Philadelphia', None), ('town lambe', 38, 'married', 'tlambeak@rediff.com', '202-931-4461', '"3 Crest Line Plaza', 'Washington', None), ('karalynn jelfs', 33, 'married', 'kjelfshq@wordpress.com', '616-732-7007', '"30 Clove Park', 'Grand Rapids', None), ('gaby haskins', 37, 'single', 'ghaskinsl7@canalblog.com', '702-717-5486', '"293 Pleasure Plaza', 'Las Vegas', None), ('blinny tattersill', 27, 'divorced', 'btattersill7r@tmall.com', '503-762-5427', '"6699 Di Loreto Avenue', 'Portland', None), ('thomas londsdale', 19, 'single', 'tlondsdalek8@ifeng.com', '716-702-8514', '"2 Cherokee Circle', 'Buffalo', None), ('britney herrieven', 38, 'married', 'bherrievenqv@ask.com', '713-204-6332', '"68027 Clyde Gallagher Hill', 'Houston', None), ('elbertine guilbert', 52, 'married', 'egu

### Establishing bigquery connection

In [ ]:
from google.oauth2 import service_account
from google.cloud import bigquery
import json

# Load the service account key file
key_path = '/content/bigquery1.json'

# Create credentials from the service account key
credentials = service_account.Credentials.from_service_account_file(key_path)

# Initialize BigQuery client with the service account credentials
project_id = 'my-project-bigquery-438808'  # Your project ID
client = bigquery.Client(credentials=credentials, project=project_id)

In [ ]:
dataset_id = 'mydataset1'
tables = client.list_tables(dataset_id)

print(f"Tables in dataset {dataset_id}:")
for table in tables:
    print(table.table_id)

Tables in dataset mydataset1:
Bq_table1
Table2
table1


In [ ]:
dataset_id = 'mydataset1'
table_id = 'Bq_table1'

query = f"SELECT * FROM `{project_id}.{dataset_id}.{table_id}`limit 20"
query_job = client.query(query)  # Run the query
bigquery_data1 = [row.values() for row in query_job.result()]
bigquery_count1 = len(bigquery_data1)

#### Bigquery table data count

In [ ]:
print(bigquery_count1)

20


In [ ]:
table_id = 'my-project-bigquery-438808.mydataset1.Bq_table1'
table = client.get_table(table_id)
bigquery_schema = table.schema
print(bigquery_schema)

[SchemaField('full_name', 'STRING', 'NULLABLE', None, None, (), None), SchemaField('age', 'FLOAT', 'NULLABLE', None, None, (), None), SchemaField('martial_status', 'STRING', 'NULLABLE', None, None, (), None), SchemaField('email', 'STRING', 'NULLABLE', None, None, (), None), SchemaField('phone', 'STRING', 'NULLABLE', None, None, (), None), SchemaField('full_address', 'STRING', 'NULLABLE', None, None, (), None), SchemaField('job_title', 'STRING', 'NULLABLE', None, None, (), None), SchemaField('membership_date', 'DATE', 'NULLABLE', None, None, (), None)]


### Schema comparissions for both Hive table and Bigquery table in strucured format

In [ ]:
import pandas as pd
# Fetch Hive table schema
def get_hive_schema(table_name):
    query = f"DESCRIBE {table_name}"
    with conn.cursor() as cursor:
        cursor.execute(query)
        schema = cursor.fetchall()
    hive_schema = pd.DataFrame(schema, columns=["column_name", "data_type", "comment"])
    return hive_schema

hive_schema = get_hive_schema("hive_table2")

In [ ]:
hive_schema

,column_name,data_type,comment
0,full_name,string,
1,age,int,
2,martial_status,string,
3,email,string,
4,phone,string,
5,full_address,string,
6,job_title,string,
7,membership_date,timestamp,


In [ ]:
# Fetch BigQuery table schema
def get_bigquery_schema(table_id):
    table = client.get_table(table_id)
    bigquery_schema = pd.DataFrame(
[(field.name, field.field_type, field.description) for field in table.schema],
        columns=["column_name", "data_type", "comment"]
    )
    return bigquery_schema

bigquery_schema = get_bigquery_schema(table_id)

In [ ]:
bigquery_schema

,column_name,data_type,comment
0,full_name,STRING,None
1,age,FLOAT,None
2,martial_status,STRING,None
3,email,STRING,None
4,phone,STRING,None
5,full_address,STRING,None
6,job_title,STRING,None
7,membership_date,DATE,None


In [ ]:
def compare_schemas(hive_schema, bigquery_schema):
    # Convert DataFrames to dictionaries for easier comparison
    hive_columns = dict(zip(hive_schema['column_name'], hive_schema['data_type']))
    bigquery_columns = dict(zip(bigquery_schema['column_name'], bigquery_schema['data_type']))

    # Compare columns in both schemas
    for column in hive_columns:
        if column in bigquery_columns:
            # Check for data type mismatch
            if hive_columns[column].upper() != bigquery_columns[column].upper():
                print(f"Data type mismatch in column '{column}': "
                      f"Hive ({hive_columns[column]}) vs BigQuery ({bigquery_columns[column]})")
        else:
            print(f"Column '{column}' exists in Hive but not in BigQuery")

    # Check if BigQuery has extra columns not in Hive
    for column in bigquery_columns:
        if column not in hive_columns:
            print(f"Column '{column}' exists in BigQuery but not in Hive")

# Usage example (assuming hive_schema and bigquery_schema are DataFrames with 'column_name' and 'data_type' columns):
compare_schemas(hive_schema, bigquery_schema)

Data type mismatch in column 'age': Hive (int) vs BigQuery (FLOAT)
Data type mismatch in column 'membership_date': Hive (timestamp) vs BigQuery (DATE)


### The data count comparission

In [ ]:
if hive_count == bigquery_count1:
  print('The data count is same in both tables')
else:
  print('The data count is not same in both tables')

The data count is not same in both tables


In [ ]:
# Close the cursor and connection
cursor.close()
conn.close()